### Paso 0: Descarga de las librerias necesarias en este proyecto

Si ya las tienes instaladas en tu entorno virtual, esta celda la puedes saltar.

In [1]:
%pip install pandas numpy scikit-learn faker tensorflow


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Paso 1: Obtención de Datos

Encontrar o construir un dataset que tenga las siguientes características:

- Que tenga al menos 5 columnas que formen parte del conjunto de variables independientes. Entre estas columnas deben encontrarse al menos:
  - **3 columnas con datos numéricos**. Al menos una de estas columnas debe tener datos ausentes.
  - **1 columna con datos categóricos de tipo nominal**.
  
- Que tenga una columna que sea la **variable dependiente** (variable objetivo o etiqueta).

### Paso 2: Preprocesamiento de Datos

Sobre dicho dataset se deberán llevar a cabo los siguientes tratamientos de preprocesado de datos:

- **Gestión de datos ausentes**: Se deberán tratar los datos ausentes de una forma razonable en términos del negocio que representen dichos datos, y en ningún caso será aceptable eliminar las filas y/o columnas que contengan datos ausentes.
  
- **Gestión de datos categóricos de tipo nominal**: Se tratarán a través de la técnica de **One Hot Encoding**.
  
- **División de los datos**: Se utilizarán el **75% de los datos** para construir el modelo y el **25% restante** para validar el modelo construido.
  
- **Normalización o estandarización**: Se normalizarán o estandarizarán (a elegir por el alumno) **todos los datos asociados a las variables independientes**.

**Nota importante**: El preprocesamiento deberá realizarse una única vez, y no una vez por cada modelo generado.

Al final, se deberá mostrar el **tiempo de ejecución** gastado en el preprocesamiento de los datos.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import faker
import random
import tensorflow as tf
from sklearn import set_config

# Función para establecer la semilla para reproducibilidad
def establecer_semilla(seed=38):
    np.random.seed(seed)         # Para NumPy
    random.seed(seed)            # Para el generador de números aleatorios de Python
    tf.random.set_seed(seed)     # Para TensorFlow, (por si acaso)
    set_config(seed)             # Para scikit-learn

# Establecer semilla para reproducibilidad
establecer_semilla(38)

# Generación de datos simulados con faker (semilla añadida aquí)
fake = faker.Faker()
fake.seed_instance(38)  # Establecemos la semilla de faker

n = 1000

data = {
    'Tamaño (m²)': [fake.random_int(30, 500) for _ in range(n)],
    'Número de Habitaciones': [fake.random_int(1, 6) for _ in range(n)],
    'Número de Baños': [fake.random_int(1, 5) for _ in range(n)],
    'Año de Construcción': [fake.random_int(1950, 2023) for _ in range(n)],
    'Ubicación': [fake.random_element(('Centro', 'Suburbio', 'Periferia')) for _ in range(n)],
    'Número de Pisos': [fake.random_int(1, 3) if fake.random_int(1, 100) > 10 else None for _ in range(n)],
    'Precio Venta': []
}

# Cálculo del precio de venta
for i in range(n):
    precio = data['Tamaño (m²)'][i] * 1000
    precio += data['Número de Habitaciones'][i] * 5000
    precio += data['Número de Baños'][i] * 3000
    precio += 10000 if data['Año de Construcción'][i] > 2000 else -5000
    precio += {'Centro': 30000, 'Suburbio': 10000, 'Periferia': 5000}[data['Ubicación'][i]]
    if data['Número de Pisos'][i] and data['Número de Pisos'][i] > 1:
        precio += 15000
    data['Precio Venta'].append(max(precio, 20000))

df = pd.DataFrame(data)
print(df.head())

# División de datos
X = df.drop('Precio Venta', axis=1)
y = df['Precio Venta']

# Preprocesamiento
num_features = ['Tamaño (m²)', 'Número de Habitaciones', 'Número de Baños', 'Año de Construcción', 'Número de Pisos']
cat_features = ['Ubicación']

num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])

# División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=38)

# Transformación
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

print(f"Datos de entrenamiento preprocesados (primeras filas):\n{X_train_preprocessed[:5]}")


   Tamaño (m²)  Número de Habitaciones  Número de Baños  Año de Construcción  \
0          357                       1                5                 1998   
1          245                       3                3                 1962   
2          249                       1                1                 1989   
3          416                       3                3                 2009   
4          400                       4                3                 1983   

   Ubicación  Número de Pisos  Precio Venta  
0     Centro              3.0        417000  
1   Suburbio              2.0        289000  
2   Suburbio              3.0        277000  
3   Suburbio              3.0        475000  
4  Periferia              NaN        429000  
Datos de entrenamiento preprocesados (primeras filas):
[[ 1.04048473  0.28702087 -0.02303029  0.71622381 -1.2636196   0.
   0.          1.        ]
 [ 0.6006097   0.87578162 -1.44027868  0.43685811  1.31373482  0.
   1.          0.        ]
 [

### Paso 3: Generación de Modelos

La práctica consistirá en generar diferentes modelos utilizando las siguientes técnicas de regresión:

#### **1. Regresión Lineal**
Se generarán los siguientes modelos:
- **Utilizando todas las variables independientes.**
- **Ignorando una de las variables independientes** (la que en apariencia sea menos relevante).
- **Ignorando dos de las variables independientes** (las que en apariencia sean menos relevantes).

#### **2. Regresión Polinómica**
Se generarán los siguientes modelos:
- **Utilizando todas las variables independientes** y generando un modelo polinómico de:
  - Grado 12
  - Grado 8
  - Grado 4
- **Ignorando una de las variables independientes** (la que en apariencia sea menos relevante) y generando un modelo polinómico de:
  - Grado 10
  - Grado 7
  - Grado 5
- **Ignorando dos de las variables independientes** (las que en apariencia sean menos relevantes) y generando un modelo polinómico de:
  - Grado 8
  - Grado 5
  - Grado 2

#### **3. Regresión de las K Valores más Cercanos (K-NN)**
Se generarán los siguientes modelos:
- **Utilizando todas las variables independientes** y generando un modelo con:
  - K = 8
  - K = 6
  - K = 4
- **Ignorando una de las variables independientes** (la que en apariencia sea menos relevante) y generando un modelo con:
  - K = 6
  - K = 4
  - K = 2
- **Ignorando dos de las variables independientes** (las que en apariencia sean menos relevantes) y generando un modelo con:
  - K = 4
  - K = 2
  - K = 1

#### **Métricas a obtener para cada modelo:**
- **Máximo Error Absoluto (M)**
- **Error Absoluto Medio (MAE)**
- **Coeficiente de Determinación (R²)**

#### **Mostrando en una tabla (por ejemplo, un DataFrame):**
- Las métricas obtenidas para cada modelo.
- El tiempo de ejecución gastado en el **entrenamiento** de cada modelo (excluido el tiempo de preprocesamiento).
- El tiempo de ejecución gastado en la **prueba** de cada modelo.


In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, max_error, r2_score
import time
import numpy as np
import pandas as pd

# Entrenar y evaluar modelos de regresión lineal
def evaluar_modelo_lineal(X_train, X_test, y_train, y_test, columnas_utilizadas):
    regressor = LinearRegression()
    
    X_train_sub = X_train[:, columnas_utilizadas] if not isinstance(X_train, pd.DataFrame) else X_train.iloc[:, columnas_utilizadas].values
    X_test_sub = X_test[:, columnas_utilizadas] if not isinstance(X_test, pd.DataFrame) else X_test.iloc[:, columnas_utilizadas].values
    
    inicio = time.time()
    regressor.fit(X_train_sub, y_train)
    tiempo_entrenamiento = time.time() - inicio

    inicio = time.time()
    y_pred = regressor.predict(X_test_sub)
    tiempo_prueba = time.time() - inicio

    return max_error(y_test, y_pred), mean_absolute_error(y_test, y_pred), r2_score(y_test, y_pred), tiempo_entrenamiento, tiempo_prueba

# Evaluación con distintas combinaciones de variables
resultados_lineales = []

todas_las_columnas = np.arange(X_train_preprocessed.shape[1])
sin_tamaño = np.delete(todas_las_columnas, 0)
sin_tamaño_habitaciones = np.delete(sin_tamaño, 0)

for desc, columnas in zip(
    ['Todas las variables', 'Sin Tamaño (m²)', 'Sin Tamaño (m²) y Número de Habitaciones'], 
    [todas_las_columnas, sin_tamaño, sin_tamaño_habitaciones]
):
    M, MAE, R2, tiempo_entrenamiento, tiempo_prueba = evaluar_modelo_lineal(X_train_preprocessed, X_test_preprocessed, y_train, y_test, columnas)
    resultados_lineales.append(['Regresión Lineal', desc, M, MAE, R2, tiempo_entrenamiento, tiempo_prueba])

# DataFrame con los resultados
resultados_df_lineales = pd.DataFrame(resultados_lineales, columns=['Modelo', 'Variables', 'Max Error', 'MAE', 'R²', 'Tiempo Entrenamiento (s)', 'Tiempo Prueba (s)'])
print(resultados_df_lineales)


             Modelo                                 Variables      Max Error  \
0  Regresión Lineal                       Todas las variables   17164.805377   
1  Regresión Lineal                           Sin Tamaño (m²)  251058.450637   
2  Regresión Lineal  Sin Tamaño (m²) y Número de Habitaciones  253780.905918   

             MAE        R²  Tiempo Entrenamiento (s)  Tiempo Prueba (s)  
0    4928.777438  0.998132                  0.011036                0.0  
1  119465.258909  0.032332                  0.002539                0.0  
2  119536.133189  0.025945                  0.004220                0.0  


In [4]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, max_error, r2_score
import time
import numpy as np
import pandas as pd

# Entrenar y evaluar modelos de regresión polinómica
def evaluar_modelo_polinomico(X_train, X_test, y_train, y_test, columnas_utilizadas, grado):
    poly_regressor = make_pipeline(PolynomialFeatures(degree=grado), LinearRegression())

    X_train_sub = X_train[:, columnas_utilizadas]
    X_test_sub = X_test[:, columnas_utilizadas]

    inicio = time.time()
    poly_regressor.fit(X_train_sub, y_train)
    tiempo_entrenamiento = time.time() - inicio

    inicio = time.time()
    y_pred_poly = poly_regressor.predict(X_test_sub)
    tiempo_prueba = time.time() - inicio

    return max_error(y_test, y_pred_poly), mean_absolute_error(y_test, y_pred_poly), r2_score(y_test, y_pred_poly), tiempo_entrenamiento, tiempo_prueba

# Evaluación con distintas combinaciones de variables y grados
resultados_polinomicos = []
todas = np.arange(X_train_preprocessed.shape[1])
sin_tamaño = np.delete(todas, 0)
sin_tamaño_habitaciones = np.delete(sin_tamaño, 0)

for desc, columnas, grados in zip(
    ['Todas las variables', 'Sin Tamaño (m²)', 'Sin Tamaño (m²) y Número de Habitaciones'],
    [todas, sin_tamaño, sin_tamaño_habitaciones],
    [[12, 8, 4], [10, 7, 5], [8, 5, 2]]
):
    for grado in grados:
        M, MAE, R2, t_ent, t_prueba = evaluar_modelo_polinomico(X_train_preprocessed, X_test_preprocessed, y_train, y_test, columnas, grado)
        resultados_polinomicos.append([f'Polinómica (grado {grado})', desc, M, MAE, R2, t_ent, t_prueba])

# DataFrame con resultados
resultados_df_polinomicos = pd.DataFrame(resultados_polinomicos, columns=['Modelo', 'Variables', 'Max Error', 'MAE', 'R²', 'Tiempo Entrenamiento (s)', 'Tiempo Prueba (s)'])
print(resultados_df_polinomicos)


                  Modelo                                 Variables  \
0  Polinómica (grado 12)                       Todas las variables   
1   Polinómica (grado 8)                       Todas las variables   
2   Polinómica (grado 4)                       Todas las variables   
3  Polinómica (grado 10)                           Sin Tamaño (m²)   
4   Polinómica (grado 7)                           Sin Tamaño (m²)   
5   Polinómica (grado 5)                           Sin Tamaño (m²)   
6   Polinómica (grado 8)  Sin Tamaño (m²) y Número de Habitaciones   
7   Polinómica (grado 5)  Sin Tamaño (m²) y Número de Habitaciones   
8   Polinómica (grado 2)  Sin Tamaño (m²) y Número de Habitaciones   

      Max Error           MAE           R²  Tiempo Entrenamiento (s)  \
0  8.847318e+05  7.288688e+04    -0.149556                 24.245872   
1  1.811744e+05  2.659235e+04     0.896614                  4.310749   
2  1.262645e+04  2.359477e+03     0.999442                  0.284267   
3  9.433253

In [5]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, max_error, r2_score
import time
import numpy as np
import pandas as pd

# Entrenar y evaluar modelos K-NN
def evaluar_modelo_knn(X_train, X_test, y_train, y_test, columnas_utilizadas, k):
    knn_regressor = KNeighborsRegressor(n_neighbors=k)

    X_train_sub = X_train[:, columnas_utilizadas]
    X_test_sub = X_test[:, columnas_utilizadas]

    inicio = time.time()
    knn_regressor.fit(X_train_sub, y_train)
    tiempo_entrenamiento = time.time() - inicio

    inicio = time.time()
    y_pred_knn = knn_regressor.predict(X_test_sub)
    tiempo_prueba = time.time() - inicio

    return max_error(y_test, y_pred_knn), mean_absolute_error(y_test, y_pred_knn), r2_score(y_test, y_pred_knn), tiempo_entrenamiento, tiempo_prueba

# Evaluación con distintas combinaciones de variables y valores de k
resultados_knn = []
todas = np.arange(X_train_preprocessed.shape[1])
sin_var1 = np.delete(todas, 0)
sin_var1_var2 = np.delete(sin_var1, 0)

for desc, columnas, ks in zip(
    ['Todas las variables', 'Sin Variable 1', 'Sin Variable 1 y 2'],
    [todas, sin_var1, sin_var1_var2],
    [[8, 6, 4], [6, 4, 2], [4, 2, 1]]
):
    for k in ks:
        M, MAE, R2, t_ent, t_prueba = evaluar_modelo_knn(X_train_preprocessed, X_test_preprocessed, y_train, y_test, columnas, k)
        resultados_knn.append([f'K-NN (k={k})', desc, M, MAE, R2, t_ent, t_prueba])

# DataFrame con resultados
resultados_df_knn = pd.DataFrame(resultados_knn, columns=['Modelo', 'Variables', 'Max Error', 'MAE', 'R²', 'Tiempo Entrenamiento (s)', 'Tiempo Prueba (s)'])
print(resultados_df_knn)


       Modelo            Variables  Max Error            MAE        R²  \
0  K-NN (k=8)  Todas las variables    97250.0   31181.875000  0.925345   
1  K-NN (k=6)  Todas las variables    92500.0   32981.666667  0.919508   
2  K-NN (k=4)  Todas las variables    99250.0   33430.000000  0.915477   
3  K-NN (k=6)       Sin Variable 1   380000.0  125309.166667 -0.104813   
4  K-NN (k=4)       Sin Variable 1   343250.0  125328.750000 -0.087349   
5  K-NN (k=2)       Sin Variable 1   388000.0  133402.500000 -0.308734   
6  K-NN (k=4)   Sin Variable 1 y 2   330500.0  124495.000000 -0.098830   
7  K-NN (k=2)   Sin Variable 1 y 2   442500.0  132455.000000 -0.374241   
8  K-NN (k=1)   Sin Variable 1 y 2   444000.0  155900.000000 -0.953939   

   Tiempo Entrenamiento (s)  Tiempo Prueba (s)  
0                  0.005939           0.000921  
1                  0.005271           0.002836  
2                  0.003609           0.001815  
3                  0.000000           0.000000  
4             

### Paso 4: Resultados

#### **1. Resultados totales**
Una tabla con todos los resultados conjuntos.

#### **2. Valoración de los resultados**
Tabla ordenadada segun la valoración dada mediante las métricas y tiempos de ejecución


In [6]:
# Crear DataFrames con la columna "Técnica" incluida directamente
def crear_df_resultados(resultados):
    df = pd.DataFrame(resultados, columns=[
        'Técnica', 'Variables Independientes', 'Max Error', 'MAE', 'R²', 'Tiempo Entrenamiento (s)', 'Tiempo Prueba (s)'
    ])
    return df

# Crear y combinar los DataFrames
df_resultados_combinados = pd.concat([
    crear_df_resultados(resultados_lineales),
    crear_df_resultados(resultados_polinomicos),
    crear_df_resultados(resultados_knn)
], ignore_index=True)

# Mostrar resultados combinados
df_resultados_combinados


,Técnica,Variables Independientes,Max Error,MAE,R²,Tiempo Entrenamiento (s),Tiempo Prueba (s)
0,Regresión Lineal,Todas las variables,1.716481e+04,4.928777e+03,0.998132,0.011036,0.000000
1,Regresión Lineal,Sin Tamaño (m²),2.510585e+05,1.194653e+05,0.032332,0.002539,0.000000
2,Regresión Lineal,Sin Tamaño (m²) y Número de Habitaciones,2.537809e+05,1.195361e+05,0.025945,0.004220,0.000000
3,Polinómica (grado 12),Todas las variables,8.847318e+05,7.288688e+04,-0.149556,24.245872,0.082831
4,Polinómica (grado 8),Todas las variables,1.811744e+05,2.659235e+04,0.896614,4.310749,0.025030
5,Polinómica (grado 4),Todas las variables,1.262645e+04,2.359477e+03,0.999442,0.284267,0.006721
6,Polinómica (grado 10),Sin Tamaño (m²),9.433253e+07,4.617267e+06,-6395.802695,4.563663,0.027991
7,Polinómica (grado 7),Sin Tamaño (m²),3.634729e+07,1.224979e+06,-602.133574,0.878581,0.006809
8,Polinómica (grado 5),Sin Tamaño (m²),4.597807e+05,1.323667e+05,-0.353530,0.220345,0.000000
9,Polinómica (grado 8),Sin Tamaño (m²) y Número de Habitaciones,3.277587e+06,2.078245e+05,-6.137792,0.671749,0.000524


In [7]:
# Valoración de los modelos generados
# Se observan las métricas y tiempos de ejecución para evaluar qué modelo es el más adecuado
df_resultados_combinados.sort_values(by=['MAE', 'R²'], ascending=[True, False])

,Técnica,Variables Independientes,Max Error,MAE,R²,Tiempo Entrenamiento (s),Tiempo Prueba (s)
5,Polinómica (grado 4),Todas las variables,1.262645e+04,2.359477e+03,0.999442,0.284267,0.006721
0,Regresión Lineal,Todas las variables,1.716481e+04,4.928777e+03,0.998132,0.011036,0.000000
4,Polinómica (grado 8),Todas las variables,1.811744e+05,2.659235e+04,0.896614,4.310749,0.025030
12,K-NN (k=8),Todas las variables,9.725000e+04,3.118188e+04,0.925345,0.005939,0.000921
13,K-NN (k=6),Todas las variables,9.250000e+04,3.298167e+04,0.919508,0.005271,0.002836
14,K-NN (k=4),Todas las variables,9.925000e+04,3.343000e+04,0.915477,0.003609,0.001815
3,Polinómica (grado 12),Todas las variables,8.847318e+05,7.288688e+04,-0.149556,24.245872,0.082831
11,Polinómica (grado 2),Sin Tamaño (m²) y Número de Habitaciones,2.770642e+05,1.168593e+05,0.044260,0.002579,0.000000
1,Regresión Lineal,Sin Tamaño (m²),2.510585e+05,1.194653e+05,0.032332,0.002539,0.000000
2,Regresión Lineal,Sin Tamaño (m²) y Número de Habitaciones,2.537809e+05,1.195361e+05,0.025945,0.004220,0.000000
